In [1]:
import tensorflow as tf
import numpy as np
import os, re, gc
from imageio import imread
from skimage import transform
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class vgg16:
    def __init__(self, imgs, weights=None, sess=None):
        self.imgs = imgs
        self.convlayers()
        
        sess.run(tf.global_variables_initializer())
        if weights is not None and sess is not None:
            self.load_weights(weights, sess)


    def convlayers(self):
        self.parameters = []

        # zero-mean input
        with tf.name_scope('preprocess') as scope:
            mean = tf.constant([123.68, 116.779, 103.939], dtype=tf.float32, shape=[1, 1, 1, 3], name='img_mean')
            
            images = self.imgs-mean
            
        # conv1_1
        with tf.name_scope('conv1_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 3, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv1_2
        with tf.name_scope('conv1_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 64], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[64], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv1_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool1
        self.pool1 = tf.nn.avg_pool(self.conv1_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool1')

        # conv2_1
        with tf.name_scope('conv2_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 64, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.pool1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv2_2
        with tf.name_scope('conv2_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 128], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[128], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv2_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool2
        self.pool2 = tf.nn.avg_pool(self.conv2_2,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool2')

        # conv3_1
        with tf.name_scope('conv3_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 128, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.pool2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_2
        with tf.name_scope('conv3_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv3_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv3_3
        with tf.name_scope('conv3_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 256], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv3_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[256], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv3_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool3
        self.pool3 = tf.nn.avg_pool(self.conv3_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool3')

        # conv4_1
        with tf.name_scope('conv4_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 256, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.pool3, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_2
        with tf.name_scope('conv4_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv4_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv4_3
        with tf.name_scope('conv4_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv4_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv4_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool4
        self.pool4 = tf.nn.avg_pool(self.conv4_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')

        # conv5_1
        with tf.name_scope('conv5_1') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.pool4, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_1 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_2
        with tf.name_scope('conv5_2') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv5_1, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_2 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # conv5_3
        with tf.name_scope('conv5_3') as scope:
            kernel = tf.Variable(tf.truncated_normal([3, 3, 512, 512], dtype=tf.float32,
                                                     stddev=1e-1), name='weights', trainable=False)
            conv = tf.nn.conv2d(self.conv5_2, kernel, [1, 1, 1, 1], padding='SAME')
            biases = tf.Variable(tf.constant(0.0, shape=[512], dtype=tf.float32),
                                 trainable=False, name='biases')
            out = tf.nn.bias_add(conv, biases)
            self.conv5_3 = tf.nn.relu(out, name=scope)
            self.parameters += [kernel, biases]

        # pool5
        self.pool5 = tf.nn.avg_pool(self.conv5_3,
                               ksize=[1, 2, 2, 1],
                               strides=[1, 2, 2, 1],
                               padding='SAME',
                               name='pool4')
        
    def load_weights(self, weight_file, sess):
        weights = np.load(weight_file)
        keys = sorted(weights.keys())

       
        for i, k in enumerate(keys):
            if k.startswith("fc"):
                continue
            else:
                print (i, k, np.shape(weights[k]), end= " ")
                sess.run(self.parameters[i].assign(weights[k]))
        print("\n")

In [17]:
class vgg16_nts(vgg16):
    def __init__(self, weights, imgs, ratio_content_to_style=None, 
                 content_act_list=None, style_act_list=None, 
                 sess=None):
        self._sess = sess
        self.graph = sess.graph
      
        vgg16.__init__(self, imgs=imgs, weights=weights, sess=sess)
        
    
       
        #activations must be in form of 
        #list_of dictionaries
        if not (content_act_list is None or ratio_content_to_style is None or style_act_list is None):
            #[{"layer_name":"conv_1_1", "content_act":value}, ...], 
            self._content_act_list = content_act_list

            #[{"layer_name":"conv_1_1", "style_act":value}, "w":0.5}...], 
            self._style_act_list = style_act_list
            self._ratio_content_to_style= ratio_content_to_style
            self.optimizer = tf.train.GradientDescentOptimizer(0.1)

          
       
           
    
    def get_content_loss(self, content, generated_act):
        return(0.5*tf.reduce_sum(tf.square(tf.subtract(content,generated_act))))
    
    def get_gram(self, content):
       
        content = tf.squeeze(content, axis=0)
        gram = tf.reshape(content, (content.shape[0].value*content.shape[1].value, content.shape[2].value))
        return(tf.matmul(tf.transpose(gram), gram))
    
    def get_style_El(self, content_gram, generated_gram, no_of_filters, filter_size):
        El = tf.reduce_sum(tf.square(tf.subtract(generated_gram, content_gram)))/(4*(no_of_filters**2)*(filter_size**2))
        return(El)
 
    def calculate_total_loss(self):
        loss_content = 0
        loss_style = 0
        
        for it in self._content_act_list:
            generated_act = self.graph.get_tensor_by_name("{}:0".format(it["layer_name"]))
            content_act = tf.constant(it["content_act"])
            loss_content += it["w"] * self.get_content_loss(content_act, generated_act)
        
        for it in self._style_act_list:
            generated_act = self.graph.get_tensor_by_name("{}:0".format(it["layer_name"]))
            content_act = tf.constant(it["style_act"])
            
            generated_gram = self.get_gram(generated_act)
            content_gram = self.get_gram(content_act)
            no_of_filters = content_act.shape[3].value
            filter_size = content_act.shape[1].value*content_act.shape[2].value
          
            
            loss_style += it["w"] * self.get_style_El(content_gram, generated_gram, no_of_filters, filter_size)
            
        return(self._ratio_content_to_style*loss_content + loss_style)
       
    def optimize(self, learning_rate=0.1):
        loss = self.calculate_total_loss()
        return([tf.train.GradientDescentOptimizer(learning_rate).minimize(loss), loss])
    

          

In [5]:
content_img = imread("sh.jpg")
content_img = transform.resize(content_img, [224, 224])
content_img = np.expand_dims(content_img, axis=0)*255.0
style_img = imread("salvador-dali-painting.jpg")
style_img = transform.resize(style_img, [224, 224])
style_img = np.expand_dims(style_img, axis=0)*255.0

C:\Users\srajabza\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [21]:
images = list()
weights_path = "vgg16_weights.npz"
#Get references

tf.reset_default_graph()
with tf.Session() as sess:
    imgs = tf.placeholder(shape=[1, 224, 224, 3], dtype=tf.float32)
    vgg_ref = vgg16_nts(imgs=imgs, weights=weights_path, sess=sess)

    style_acts = [{"layer_name":"conv1_1", "style_act":0, "w":1/5},
                  {"layer_name":"conv2_1", "style_act":0, "w":1/5},
                  {"layer_name":"conv3_1", "style_act":0, "w":1/5},
                  {"layer_name":"conv4_1", "style_act":0, "w":1/5},
                  {"layer_name":"conv5_1", "style_act":0, "w":1/5}]
    
    content_acts = [{"layer_name":"conv4_2", "content_act":0, "w":1}]
    
    for i, ls in enumerate(style_acts):
        style_acts[i]["style_act"] = sess.run(ls["layer_name"]+":0", feed_dict={imgs:style_img})
        
    for i, lc in enumerate(content_acts):
        content_acts[i]["content_act"] = sess.run(lc["layer_name"]+":0", feed_dict={imgs:content_img})
        

tf.reset_default_graph()
with tf.Session() as sess:
    imgs = tf.get_variable(shape=[1, 224, 224, 3], initializer=tf.random_uniform_initializer(0, 255), 
                           name="painted", dtype=tf.float32) 
    
    sess.run(tf.variables_initializer([imgs]))
    vgg_paint = vgg16_nts(imgs=imgs, weights=weights_path, 
                          ratio_content_to_style=1e-1, style_act_list=style_acts, 
                          content_act_list=content_acts, sess=sess)
    
    
    for epoch in range(1000):
        _, ls = sess.run(vgg_paint.optimize())
        if epoch%10==0:
            images.append(sess.run(imgs))
            
        print("\r", ls, end=" ")
 

0 conv1_1_W (3, 3, 3, 64) 1 conv1_1_b (64,) 2 conv1_2_W (3, 3, 64, 64) 3 conv1_2_b (64,) 4 conv2_1_W (3, 3, 64, 128) 5 conv2_1_b (128,) 6 conv2_2_W (3, 3, 128, 128) 7 conv2_2_b (128,) 8 conv3_1_W (3, 3, 128, 256) 9 conv3_1_b (256,) 10 conv3_2_W (3, 3, 256, 256) 11 conv3_2_b (256,) 12 conv3_3_W (3, 3, 256, 256) 13 conv3_3_b (256,) 14 conv4_1_W (3, 3, 256, 512) 15 conv4_1_b (512,) 16 conv4_2_W (3, 3, 512, 512) 17 conv4_2_b (512,) 18 conv4_3_W (3, 3, 512, 512) 19 conv4_3_b (512,) 20 conv5_1_W (3, 3, 512, 512) 21 conv5_1_b (512,) 22 conv5_2_W (3, 3, 512, 512) 23 conv5_2_b (512,) 24 conv5_3_W (3, 3, 512, 512) 25 conv5_3_b (512,) 

0 conv1_1_W (3, 3, 3, 64) 1 conv1_1_b (64,) 2 conv1_2_W (3, 3, 64, 64) 3 conv1_2_b (64,) 4 conv2_1_W (3, 3, 64, 128) 5 conv2_1_b (128,) 6 conv2_2_W (3, 3, 128, 128) 7 conv2_2_b (128,) 8 conv3_1_W (3, 3, 128, 256) 9 conv3_1_b (256,) 10 conv3_2_W (3, 3, 256, 256) 11 conv3_2_b (256,) 12 conv3_3_W (3, 3, 256, 256) 13 conv3_3_b (256,) 14 conv4_1_W (3, 3, 256, 512) 15 c

KeyboardInterrupt: 

In [ ]:
def print_suc_msg(s):
    print("{} Passed".format(s))


tf.reset_default_graph()
with tf.Session() as sess:
    imgs = tf.placeholder(tf.float32,[1, 224, 224, 3])
    vgg = vgg16_nts(imgs, weights="vgg16_weights_scaled.npz", sess=sess)

    #################
    #Testing content#
    #################

    np.random.seed(0)
    content_act = np.random.uniform(size=(10, 10)).astype(np.float32) - 0.5
    np.random.seed(1)
    generated_act = np.random.uniform(size=(10, 10)).astype(np.float32) - 0.5
    assert(np.sum((content_act - generated_act)**2)/2 == 
           sess.run(vgg.get_content_loss(tf.constant(content_act), tf.constant(generated_act))))

    print_suc_msg("Testing Content")


    ##############
    #Testing gram#
    ##############

    np.random.seed(0)
    content_act = np.random.uniform(size=(1, 5, 5, 10)).astype(np.float32) - 0.5

    g_content = sess.run(vgg.get_gram(tf.constant(content_act)))
    assert(np.abs(g_content[2,8] - np.sum(np.multiply(content_act[0,:,:,2].ravel(),content_act[0,:,:,8].ravel())))<1e-4)
    assert(np.abs(g_content[4,5] - np.sum(np.multiply(content_act[0,:,:,4].ravel(),content_act[0,:,:,5].ravel())))<1e-4)
    print_suc_msg("Testing Gram")

    ###############
    #Testing Style#
    ###############

    np.random.seed(0)
    content_act = np.random.uniform(size=(1, 5, 5, 10)).astype(np.float32) - 0.5
    np.random.seed(1)
    generated_act = np.random.uniform(size=(1, 5, 5, 10)).astype(np.float32) - 0.5

    el_style = sess.run(vgg.get_style_El(content_gram=vgg.get_gram(tf.constant(content_act)), 
                                generated_gram=vgg.get_gram(tf.constant(generated_act)),
                                no_of_filters=content_act.shape[3], 
                                filter_size=content_act.shape[2]*content_act.shape[1]))

    assert(el_style - np.sum(np.square(sess.run(vgg.get_gram(tf.constant(content_act))) - 
                                       sess.run(vgg.get_gram(tf.constant(generated_act)))))/(4*(25**2)*100)<1e-4)

       


In [23]:
plt.imshow(images[0].squeeze()/255.0)

IndexError: list index out of range